In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")

In [ ]:
from tabulate import tabulate

data_info=[[i,df[i].dtypes, df[i].count(), df[i].max()] for i in df.columns]

print(tabulate(data_info, headers=["Column", "Type", "Count", "Max"]))

Öncelikle elimizde bulunan datayı tanımak istiyorum. Bunun için **describe**() fonksiyonunu kullanıyorum. 

In [ ]:
df.describe()

**scientific notation** olarak verilerin sergilenmesi nedeni ile bu tablo benim için anlaşılır değil o nedenle bu tabloyu biraz düzenliyorum. 

In [ ]:
df.describe().apply(lambda x: x.apply(lambda s :"{:.2f}".format(s)))

In [ ]:
df.head(5) # ilk 5 Satır

In [ ]:
df.tail(5)

Timestamp sütununda bulunan zaman bilgisini anlayamadığım için yeni bir sütunlar oluşturup orada anlaşılır zaman bilgisini tutmak istiyorum. 

In [ ]:
df["Time_year"]=df["Timestamp"].apply(lambda x: datetime.fromtimestamp(x).date().year)
df["Time_month"]=df["Timestamp"].apply(lambda x: datetime.fromtimestamp(x).date().month)
df["Time_day"]=df["Timestamp"].apply(lambda x: datetime.fromtimestamp(x).date().day)
df["Time_hour"]=df["Timestamp"].apply(lambda x: datetime.fromtimestamp(x).time())
df["Time"]=df["Timestamp"].apply(lambda x: datetime.fromtimestamp(x))
df

**Not**: Timestamp sütununda bulunan veriyi anlaşlır tarih şeklinde Time başlıklı bir sütunda yeniden oluşturunca, 31 Aralık 2011 sabahı saat 07:52 den , 31 Mart 2021 saat 00:00'a kadar her dakika oluşturulmış kayıtlardan oluştuğunu anlıyorum.    

açılış ve kapanış arasındaki artış ve azalış trendinin volume nasıl bir katkısı var merak ediyorum. 

In [ ]:
df["turev_price"]=(df["Close"]-df["Open"])/df["Open"]
df.head(10)

Bazı satırların NaN olarak kayıt edildiğini görüyorum yani hiç işlem görmemiş olan zamanlar var. Daha temiz bir tablo için işlem görmemiş anları tablodan temizlemek yani tablodan çıkarmak istiyorum

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

Tablomuzu temizledik ve anlayacağımız şekilde düzenledik şimdi bunu artık görselleştirebiliriz. 

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(1,1,1)
plt.scatter(df["Time"], ((df["High"]-df["Low"])/df["High"]))
plt.ylabel("En yüksek değer ile en düşük değerin farkının, en yüksek değere oranı.", fontsize=12)
plt.xlabel("Zaman", fontsize=12)
plt.title("BTC değerinin en aynı gün içinde en yüksek ve en düşük değerlerinin oranı", fontsize = 16)
plt.show()

Yukarıdaki tabloya göre, 2013,2014,2015 yıllarında aynı gün içerisinde BTC'nin değeri en dip ve en tepe noktası arasındaki fark çok fazla olmuş ve sonrasında daha sakin bir süreç yaşamış.

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(1,1,1)
plt.scatter(df["Time"], ((df["High"]-df["Low"])/df["High"]))
plt.ylabel("En yüksek değer ile en düşük değerin farkının, en yüksek değere oranı.", fontsize=12)
plt.xlabel("Zaman", fontsize=12)
plt.title("BTC değerinin en aynı gün içinde en yüksek ve en düşük değerlerinin oranı", fontsize = 16)
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot( df["Time"], df["Open"],color="green", label="Open / Time")
plt.legend(loc="upper left")
plt.xlabel("Timestamp")
plt.ylabel("Open Value")
plt.title("BTC OPEN değerleri ", fontsize=16)
plt.show()

BTC, yukarıdaki tabloya göe 2018 yılına kadar pek ciddi artış göstermezken 2018 dan itibaren değeri hızla artmıştır. 

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(1,1,1)
sns.barplot(x=df["Time_year"],y=df["Open"], hue=df["Time_month"])
plt.title("BTC OPEN değerleri ", fontsize=16)
plt.show()


Yukarıdaki tabloya göre, 2014 ve 2018 yılı düşüş ile geçerken, bunun dışında genelde yükseliş ile geçmiştir. özellikle 2020 yıılı Ekim ayından sonra hızlı artış görünüyor.  


In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(1,1,1)
plt.scatter(df["Time"],df["Volume_(BTC)"])
plt.plot(df["Time"],df["Open"], color="red")
plt.title("BTC OPEN değerleri ile işlem hacminin ilşkisi", fontsize=16)
plt.show()

Bu tabloda, gerçekleşen işlem hacmi görünüyor ve 2017 yılına kadar çok yoğın işlem hacmi olurken 2017 den sonra işlem hacim daha düşük ve sakin bir güzerhgah izlemiş durumda. df["Open"] değerlerine bakarak yükselen BTC değeri nedeni ile işlem hacminin düştüğünü düşünüyorum. 

In [ ]:
plt.figure(figsize=(30,10))
df["High"].plot(kind="line", x=df["Time_year"], y=df["High"],  color="black", label="High", linewidth=0.5, alpha=1, grid=True, linestyle="-")
df["Low"].plot(kind="line", x=df["Time_year"], y=df["Low"], color="red",label="Low", linewidth=0.5)
plt.legend(loc="upper left")
plt.ylabel("High and Low")
plt.xlabel("x Axis")
plt.title("BTC high and Low Değerleri", fontsize=16)
plt.show()


In [ ]:
df.corr()

In [ ]:
plt.subplots(figsize=(18,18))
sns.heatmap(df.corr(), annot=True, linewidth=.5,fmt=".1f")
plt.show()

Yukarıdaki Correlation tablosun da görünuyor ki, Open, Close, Low ve high değerleri arasında doğru orantı var. 

In [ ]:
k=0
for i, value in df.iterrows():   
    if value["Open"]>=0:
        k+=1
        if k==2:
            dt_obj=datetime.fromtimestamp(value["Timestamp"])
            print("ilk olarak, {}. satırda {} anında haraket görülmüştür.".format(i,dt_obj))
            print("{} anında ki değer: {}".format(dt_obj,value["Open"]))          
            break

In [ ]:
x= dict(df[df["Open"]==df["Open"].max()])
y= dict(df[df["Open"]==df["Open"].min()])
w=pd.DataFrame(df[df["Open"]==df["Open"].max()])
zdf=pd.DataFrame(df[df["Open"]==df["Open"].min()])


#print(len(x))
#print(len(y))
#print(x)
print(datetime.fromtimestamp(x["Timestamp"]))
print(w)
print(zdf)

for i, value in zdf.iterrows():
    print("min değer ilk gördüğümüz an :" , datetime.fromtimestamp(value.Timestamp))
    break

In [ ]:
df_max = df[df["Open"]==df["Open"].max()]

print("Max. Açılış Tarihi: ", datetime.fromtimestamp(df_max["Timestamp"]))
print("Ulaşılan maksimum açılış derğeri : ", df["Open"].max()) 
